In [5]:
# Paketleri yükle
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch, cm
from reportlab.graphics.shapes import *
from pandas import read_excel
import copy
from PyPDF2 import PdfReader, PdfWriter
import io
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import png
import qrcode
from PIL import Image
import qrcode.image.pil

In [3]:
# DejaVu Sans fontunun kaydedilmesi:
font_path = r"C:/Users/esrab/PythonProject/ReportLab/dejavu-fonts-ttf-2.37/ttf/DejaVuSans.ttf"
pdfmetrics.registerFont(TTFont('DejaVu', font_path))

In [16]:
participant_list = read_excel("./Badge_list.xlsx", sheet_name="Badge_list")  
participant_list

,No,Title,Name,Affiliation,Type
0,144,Mrs.,NAZLI CAN,İstanbul University,Participant
1,203,Assist. Prof. Dr.,ESRA GÜR,Istanbul University,Listener
2,205,Prof. Dr.,ÖZCAN AKBULUT,Sakarya University,Honorary Chair
3,207,Mr.,BURAK AYDIN,Düzce University,Participant


In [17]:
packet = io.BytesIO()
cart_size = (5.001 * cm, 8.493 * cm)
canvas_obj = canvas.Canvas(packet, pagesize=cart_size)

for index, row in participant_list.iterrows():
    no, title, name, uni, participantType = row['No'], row['Title'], row['Name'], row['Affiliation'], row['Type']
    print(f"No: {no}, Title: {title}, Name: {name}, University: {uni}, Participant Type: {participantType}")


    qr = qrcode.QRCode(version=1, box_size=1.5, border=2)
    qr.add_data("{} {} {} {} {}".format(no, title, name, uni, participantType))
    qr.make(fit=True)
    img = qr.make_image(image_factory=qrcode.image.pil.PilImage, fill_color='black', back_color='white')
    img = img.resize((64, 64), Image.Resampling.LANCZOS)
    img.save('qrcode{}.png'.format(no))
    canvas_obj.drawImage('qrcode{}.png'.format(no), 1.24*cm, 2.45*cm)
    canvas_obj.showPage()

canvas_obj.save()


No: 144, Title: Mrs., Name: NAZLI CAN, University: İstanbul University, Participant Type: Participant
No: 203, Title: Assist. Prof. Dr., Name: ESRA GÜR, University: Istanbul University, Participant Type: Listener
No: 205, Title: Prof. Dr. , Name: ÖZCAN AKBULUT, University: Sakarya University, Participant Type: Honorary Chair
No: 207, Title: Mr., Name: BURAK AYDIN, University: Düzce University, Participant Type: Participant


In [18]:
packet.seek(0)
canvas_pdf = PdfReader(packet)
badge_pdf = PdfReader(open("Badge.pdf", 'rb'))

output = PdfWriter()
for i in range(len(canvas_pdf.pages)):
    page = copy.copy(badge_pdf.pages[0])
    page.merge_page(canvas_pdf.pages[i])
    output.add_page(page)
    print(f"{i + 1}. sertifika oluşturuldu.")  


with open("badge_created.pdf", "wb") as outfile:  
    output.write(outfile) 

1. sertifika oluşturuldu.
2. sertifika oluşturuldu.
3. sertifika oluşturuldu.
4. sertifika oluşturuldu.
